In [1]:
#Importing Modules used for the Data Cleaning & Model Building
import pandas as pd
import numpy as np
import os
import random
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
import keras

In [3]:
#Initialize the path for the files to be pulled from and also the desired columns
path = "Hemorrhage Segmentation Project"
file_list = os.listdir(path)
file_list.remove('Segmentation Glossary (1).rtf')
file_list.remove('flagged.txt')
file_list.remove('hemorrhage-labels.csv')
file_list.remove('.ipynb_checkpoints')
column_names = ["Origin","Labeling State","All Labels","Correct Label","Type"]
df_raw = pd.DataFrame(columns=column_names)

#This Loop opens each file in the directory and creates a combined dataframe with the file names and type for each segmented file
for file in file_list:
    cols = column_names[:4]
    df_temp = pd.read_csv(path + "/" + file)
    df_temp = df_temp[cols]
    df_temp["Type"] = file.strip().split()[0].replace("Results_","")
    df_temp
    df_raw = df_raw.append(df_temp)

#Remove abnormalilites from the Correct Label section
df = df_raw[df_raw["Labeling State"] == "Labeled"].dropna().reset_index().drop(['index'],axis = 1)
df['Valid'] = df['Correct Label'].apply(lambda x: 1 if "x" in x else 0)
df = df[df.Valid == 1].reset_index().drop(["index","Valid"],axis = 1)

/var/folders/6g/lht9tm8x1bz3sdrs0xk_d2m00000gn/T/ipykernel_20776/1349658628.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append(df_temp)
/var/folders/6g/lht9tm8x1bz3sdrs0xk_d2m00000gn/T/ipykernel_20776/1349658628.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append(df_temp)
/var/folders/6g/lht9tm8x1bz3sdrs0xk_d2m00000gn/T/ipykernel_20776/1349658628.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append(df_temp)
/var/folders/6g/lht9tm8x1bz3sdrs0xk_d2m00000gn/T/ipykernel_20776/1349658628.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw = df_raw.append(df_temp)


In [4]:
df['Type'].unique()

array(['Subdural', 'Intraparenchymal', 'Multiple', 'Epidural',
       'Subarachnoid'], dtype=object)

In [5]:
#Assemble a random sample of normal images to to add to the training list
N = 5000
path_n = 'normal/max_contrast_window'
file_list_n = os.listdir(path_n)

random.seed(123) #Keeps random state the same while we are testing 
normal_files = random.sample(file_list_n,k = N)

#Format df same as the other data frame so we can easily append it
df_normal = pd.DataFrame(normal_files,columns= ['Origin'])
df_normal["Labeling State"] = "None"
df_normal["All Labels"] = "None"
df_normal["Correct Label"] = "None"
df_normal["Type"] = "Normal"

#Append this Normal File df to the original df
df = df.append(df_normal).reset_index().drop(["index"],axis = 1)
df = df[df['Origin']!='.ipynb_checkpoints']

# replacing values
df['Type'].replace(['Normal', 'Subdural', 'Intraparenchymal', 'Multiple', 'Epidural', 'Subarachnoid'],
                   [0, 1, 2, 3, 4, 5], inplace=True)
df

/var/folders/6g/lht9tm8x1bz3sdrs0xk_d2m00000gn/T/ipykernel_20776/1372997237.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_normal).reset_index().drop(["index"],axis = 1)


,Origin,Labeling State,All Labels,Correct Label,Type
0,ID_0061cb462.jpg,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...","[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",1
1,ID_0061cb462.jpg,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...","[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",1
2,ID_00eb6f7cc.jpg,Labeled,"['[{""x"": 0.48163298, ""y"": 0.71568763}, {""x"": 0...","[[{'x': 0.447265625, 'y': 0.73828125}, {'x': 0...",1
3,ID_0162f0acb.jpg,Labeled,"['[{""x"": 0.44486272, ""y"": 0.4724329}, {""x"": 0....","[[{'x': 0.46875, 'y': 0.79296875}, {'x': 0.468...",1
4,ID_029b94665.jpg,Labeled,"['[{""x"": 0.7883911, ""y"": 0.4989282}, {""x"": 0.7...","[[{'x': 0.55859375, 'y': 0.32421875}, {'x': 0....",1
...,...,...,...,...,...
7265,ID_00f75c05a.jpg,None,None,None,0
7266,ID_0247a8b08.jpg,None,None,None,0
7267,ID_03598c68f.jpg,None,None,None,0
7268,ID_02bd91941.jpg,None,None,None,0


In [6]:
#list of indices with wrong img shapes - remove from original df and from img df
wrong_list = []

#empty array for image data
data = np.zeros([7269, 512, 512])

for i, file_name in enumerate(df.Origin):
    img = np.mean(matplotlib.image.imread('relevant_images/' + file_name),axis=2)#.reshape(-1)
    if img.shape[0] == 512:
        data[i,:,:] = img
    else:
        wrong_list.append(i)
    
data = np.delete(data, wrong_list, 0)
df = df.drop(wrong_list, axis=0)

In [7]:
X_train_full, X_test, y_train_full, y_test = train_test_split(data, df["Type"], random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[512, 512]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(25, activation="relu"),
    keras.layers.Dense(6, activation='softmax')
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history = model.fit(X_train, y_train,
          batch_size=128, epochs=10,
          verbose=1,
          validation_data=(X_valid, y_valid))
# model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
# history = model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))
# mse_test = model.evaluate(X_test, y_test)
model.evaluate(X_test, y_test)

2022-11-27 19:33:15.239387: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
32/32 [==============================] - 56s 1s/step - loss: nan - accuracy: 0.6517 - val_loss: nan - val_accuracy: 0.7041
Epoch 2/10
32/32 [==============================] - 51s 2s/step - loss: nan - accuracy: 0.6892 - val_loss: nan - val_accuracy: 0.7041
Epoch 3/10
32/32 [==============================] - 37s 1s/step - loss: nan - accuracy: 0.6892 - val_loss: nan - val_accuracy: 0.7041
Epoch 4/10
32/32 [==============================] - 45s 1s/step - loss: nan - accuracy: 0.6892 - val_loss: nan - val_accuracy: 0.7041
Epoch 5/10
32/32 [==============================] - 30s 959ms/step - loss: nan - accuracy: 0.6892 - val_loss: nan - val_accuracy: 0.7041
Epoch 6/10
32/32 [==============================] - 38s 1s/step - loss: nan - accuracy: 0.6892 - val_loss: nan - val_accuracy: 0.7041
Epoch 7/10
32/32 [==============================] - 44s 1s/step - loss: nan - accuracy: 0.6892 - val_loss: nan - val_accuracy: 0.7041
Epoch 8/10
32/32 [==============================] - 31s 937

[nan, 0.6734581589698792]